Imports


In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.object_detection import non_max_suppression

import numpy as np
import argparse
import time
import cv2
import imutils

Minimum Confidence

In [ ]:
min_conf = 0.5

Video Path

In [ ]:
vidPath = ""

Prediction Function

In [ ]:
def decode_pred(scores, geo):
    (numR, numC) = scores.shape[2:4]
    rects = []
    conf = []

    for y in range(0, numR):
        scoresData = scores[0, 0, y]
        x0Data = geo[0, 0, y]
        x1Data = geo[0, 1, y]
        x2Data = geo[0, 2, y]
        x3Data = geo[0, 3, y]
        angleData = geo[0, 4, y]

        for x in range(0, numC):
            if scoresData[x] < min_conf:
                continue

            (offsetX, offsetY) = (4.0 * x, 4.0 * y)

            angle = angleData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            h = x0Data[x] + x2Data[x]
            w = x1Data[x] + x3Data[x]

            endX = int(offsetX + (cos * x1Data[x]) + (sin * x2Data[x]))
            endY = int(offsetY - (sin * x1Data[x]) + (cos * x2Data[x]))

            startX = int(endX - w)
            startY = int(endY - h)

            rects.append((startX, startY, endX, endY))
            conf.append(scoresData[x])

Parsing Args

In [ ]:
ap = argparse.ArgumentParser()

ap.add_argument("-east", "--east", type=str, required=True, help="path to input EAST text detector")
ap.add_argument("-v", "--video", type=str, help="path to optinal input video file")
ap.add_argument("-c", "--min-confidence", type=float, default=0.5, help="minimum probability required to inspect a region")
ap.add_argument("-w", "--width", type=int, default=320, help="resized image width (should be multiple of 32)")
ap.add_argument("-e", "--height", type=int, default=320, help="resized image height (should be multiple of 32)")

args = vars(ap.parse_args(args=[]))

Initializations

In [ ]:
(W, H) = (None, None)
(newW, newH) = (args["width"], args["height"])
(rW, rH) = (None, None)

layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]

print("[INFO] loading EAST")
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

Video Stream

In [ ]:
if vidPath == "":
    print("[INFO] Starting WebCam")
    vs = VideoStream(src = 0).start()
else:
    va = cv2.VideoCapture(args["video"])

fps = FPS().start()
    

Looping Video Frames

In [ ]:
while True:
    frame = vs.read()
    frame = frame[1] if args.get("video", False) else frame

    if frame is None:
        break

    frame = imutils.resize(frame, width = 1000)
    orig = frame.copy()

    if W is None or H is None:
        (H, W) = frame.shape[:2]
        rW = W / float(newW)
        rH = H/ float(newH)

    frame = cv2.resize(frame, (newW, newH))